In [1]:
from keras.applications import VGG16
img_rows = 224
img_cols = 224
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


In [2]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [3]:
for layer in model.layers:
    layer.trainable = False

In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [5]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [6]:
model.layers

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 4

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Faces/train/'
validation_data_dir = 'Faces/validation/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

Found 1065 images belonging to 4 classes.
Found 412 images belonging to 4 classes.


In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("faces_VGG16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 10
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator)

Epoch 1/10
1065/1065 [==============================] - 1314s 1s/step - loss: 0.8700 - accuracy: 0.7362 - val_loss: 1.4554e-04 - val_accuracy: 0.9879

Epoch 00001: val_loss improved from inf to 0.00015, saving model to faces_VGG16.h5
Epoch 2/10
1065/1065 [==============================] - 1282s 1s/step - loss: 0.3345 - accuracy: 0.9136 - val_loss: 3.5763e-07 - val_accuracy: 0.9927

Epoch 00002: val_loss improved from 0.00015 to 0.00000, saving model to faces_VGG16.h5
Epoch 3/10
1065/1065 [==============================] - 1362s 1s/step - loss: 0.2494 - accuracy: 0.9286 - val_loss: 0.0000e+00 - val_accuracy: 0.9927

Epoch 00003: val_loss improved from 0.00000 to 0.00000, saving model to faces_VGG16.h5
Epoch 4/10
1065/1065 [==============================] - 1281s 1s/step - loss: 0.1746 - accuracy: 0.9587 - val_loss: 0.0000e+00 - val_accuracy: 0.9951

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/10
1065/1065 [==============================] - 3616s 3s/step - loss: 0.1825 - a

In [12]:
train_generator.class_indices

{'Ashish': 0, 'Poorvi': 1, 'Pushpa': 2, 'Shobha': 3}